In [113]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt

In [114]:
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10

In [115]:
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

# This function takes dataset containing pairs and 
# returns a dataframe of n*21 dimention containing their image specifications 
# It performs feature concatination, i.e. It takes two sets of 9 features for each image
# and concats them into 18 features

def merge_data_set_feature_concatenation(image_specs, pair_set):
    df3 = pd.merge(pair_set, image_specs ,left_on = "img_id_A", right_on = "img_id",how="inner")
    df4 = pd.merge(df3, image_specs ,left_on = 'img_id_B', right_on = 'img_id',how="inner")
    
    targets = df4['target']

    _RawTarget = targets.values
    datasetStriped = df4.drop(['img_id_A', 'img_id_B', 'target','img_id_x', 'img_id_y'], axis=1)
    
    uniques = datasetStriped.apply(lambda x: x.nunique())
    datasetStriped = datasetStriped.drop(uniques[uniques==1].index, axis=1)
    
    _RawData = np.transpose(datasetStriped.values)
    
    return _RawTarget, _RawData

# This function takes dataset containing pairs and 
# returns a dataframe of n*12 dimention containing their image specifications 
# It performs feature subtraction, i.e. It takes two sets of 9 features for each image
# and subtracts them and return 9 features

def merge_data_set_feature_subtraction(image_specs, pair_set):
    targets = pair_set['target']
    
    df3 = pd.merge(pair_set, image_specs ,left_on = "img_id_A", right_on = "img_id",how="inner")
    df3 = df3.drop(['img_id_A', 'img_id_B', 'target','img_id'], axis=1)
    

    df4 = pd.merge(pair_set, image_specs ,left_on = 'img_id_B', right_on = 'img_id',how="inner")
    df4 = df4.drop(['img_id_A', 'img_id_B', 'target','img_id'], axis=1)
    
    df5 = df3.sub(df4)
    df5 = df5.abs()
    
    uniques = df5.apply(lambda x: x.nunique())
    df5 = df5.drop(uniques[uniques==1].index, axis=1)
    
    temp_np = df5.values
    

    _RawTarget = targets.values
    _RawData = np.transpose(temp_np)
    
    return _RawTarget, _RawData

In [116]:
def generate_training_validation_testing_ds(RawTarget, RawData):
    _TrainingTarget = GenerateTrainingTarget(RawTarget,TrainingPercent)
    _TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
    print("Training Target: "+ str(_TrainingTarget.shape))
    print("Training Data: "+ str(_TrainingData.shape))

    _ValDataAct = GenerateValTargetVector(RawTarget,ValidationPercent, (len(_TrainingTarget)))
    _ValData    = GenerateValData(RawData,ValidationPercent, (len(_TrainingTarget)))
    print("Validation Target: "+ str(_ValDataAct.shape))
    print("Validation Data: "+ str(_ValData.shape))

    _TestDataAct = GenerateValTargetVector(RawTarget,TestPercent, (len(_TrainingTarget)+len(_ValDataAct)))
    _TestData = GenerateValData(RawData,TestPercent, (len(_TrainingTarget)+len(_ValDataAct)))
    print("Testing Target: "+ str(_TestDataAct.shape))
    print("Testing Data: "+ str(_TestData.shape))
    
    return _TrainingTarget, _TrainingData, _ValDataAct, _ValData, _TestDataAct, _TestData

In [117]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [118]:
def execute_logistic_regression(_loop_range, _TrainingTarget, _TrainingData, _ValDataAct, _ValData, _TestDataAct, _TestData):
    W = np.ones(_TrainingData.shape[0])
    W_NOW = np.transpose(W)
    learning_rate = 0.01
    L_Erms_Val   = []
    L_Erms_TR    = []
    L_Erms_Test  = []
    L_Acc_Val   = []
    L_Acc_TR    = []
    L_Acc_Test  = []

    for i in range(_loop_range):

        z = np.dot(W_NOW, _TrainingData)
        predicted_values = sigmoid(z)

        diff = np.subtract(predicted_values, _TrainingTarget) #need to check np.substract

        Delta_W = np.dot(_TrainingData, diff)/_TrainingTarget.shape[0]

        W_NOW = W_NOW - (learning_rate*Delta_W)

            #-----------------TrainingData Accuracy---------------------#
        TR_TEST_OUT   = sigmoid(np.dot(W_NOW, _TrainingData))
        Erms_TR       = GetErms(TR_TEST_OUT, _TrainingTarget)
        L_Acc_TR.append(float(Erms_TR.split(',')[0]))
        L_Erms_TR.append(float(Erms_TR.split(',')[1]))

        #-----------------ValidationData Accuracy---------------------#
        VAL_TEST_OUT  = sigmoid(np.dot(W_NOW, _ValData))
        Erms_Val      = GetErms(VAL_TEST_OUT, _ValDataAct)
        L_Acc_Val.append(float(Erms_Val.split(',')[0]))
        L_Erms_Val.append(float(Erms_Val.split(',')[1]))

        #-----------------TestingData Accuracy---------------------#
        TEST_OUT      = sigmoid(np.dot(W_NOW, _TestData))
        Erms_Test = GetErms(TEST_OUT, _TestDataAct)
        L_Acc_Test.append(float(Erms_Test.split(',')[0]))
        L_Erms_Test.append(float(Erms_Test.split(',')[1]))

    print ('----------Gradient Descent Solution--------------------')
    print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
    print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
    print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
    print ("Accuracy Training   = " + str(np.around(max(L_Acc_TR),5)))
    print ("Accuracy Validation = " + str(np.around(max(L_Acc_Val),5)))
    print ("Accuracy Testing    = " + str(np.around(max(L_Acc_Test),5)))

# Human Observed Data

In [119]:
image_specs_df = pd.read_csv("HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv", index_col=0)

same_pair_df = pd.read_csv("HumanObserved-Dataset/HumanObserved-Features-Data/same_pairs.csv")

diff_pair_df = pd.read_csv("HumanObserved-Dataset/HumanObserved-Features-Data/diffn_pairs.csv")


# create a dataframe by taking a subset from samples of different writers
# Since unmatched dataset is huge we are randomly creating a sample with same data size (791 rows)
# because if the amount of unmatched data overwhelms the matched data, the model may overfit

diff_pair_df_sample = diff_pair_df.sample(n=same_pair_df.shape[0], replace=True)

#Merging same and different writer's data set into one. (1582 rows)
dataset_pairs = pd.concat([diff_pair_df_sample,same_pair_df]).sample(frac=1).reset_index(drop=True)

In [120]:
# Now we are merging the image specifications with the dataset, we will use this dataset to train
# test or validate data.

RawTarget, RawData = merge_data_set_feature_concatenation(image_specs_df, dataset_pairs)
print("=========================")
print("== Human Observed Data ==")
print("==== Concatenation ======")
print("=========================")
print("RawData : " + str(RawData.shape))
print("RawTarget : " + str(RawTarget.shape))

TrainingTarget, TrainingData, ValDataAct, ValData, TestDataAct, TestData = generate_training_validation_testing_ds(
                                                                            RawTarget, RawData)

execute_logistic_regression(1000, TrainingTarget, TrainingData, ValDataAct, ValData, TestDataAct, TestData)



== Human Observed Data ==
==== Concatenation ======
RawData : (18, 1582)
RawTarget : (1582,)
Training Target: (1266,)
Training Data: (18, 1266)
Validation Target: (158,)
Validation Data: (18, 158)
Testing Target: (157,)
Testing Data: (18, 157)
----------Gradient Descent Solution--------------------
E_rms Training   = 0.49421
E_rms Validation = 0.50435
E_rms Testing    = 0.49684
Accuracy Training   = 55.52923
Accuracy Validation = 60.75949
Accuracy Testing    = 58.59873


In [121]:
# Now we are merging the image specifications with the dataset, we will use this dataset to train
# test or validate data.

RawTarget, RawData = merge_data_set_feature_subtraction(image_specs_df, dataset_pairs)
print("=========================")
print("== Human Observed Data ==")
print("==== Subtraction ======")
print("=========================")
print("RawData : " + str(RawData.shape))
print("RawTarget : " + str(RawTarget.shape))

TrainingTarget, TrainingData, ValDataAct, ValData, TestDataAct, TestData = generate_training_validation_testing_ds(
                                                                            RawTarget, RawData)

execute_logistic_regression(1000, TrainingTarget, TrainingData, ValDataAct, ValData, TestDataAct, TestData)

== Human Observed Data ==
==== Subtraction ======
RawData : (9, 1582)
RawTarget : (1582,)
Training Target: (1266,)
Training Data: (9, 1266)
Validation Target: (158,)
Validation Data: (9, 158)
Testing Target: (157,)
Testing Data: (9, 157)
----------Gradient Descent Solution--------------------
E_rms Training   = 0.50591
E_rms Validation = 0.49828
E_rms Testing    = 0.50724
Accuracy Training   = 51.18483
Accuracy Validation = 58.86076
Accuracy Testing    = 56.05096
